# **Common parts for end point prediction and trajectory completion**

## **Connect to google drive and download dataset**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from google.colab import auth
auth.authenticate_user()
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse
!mkdir waymo_dataset
!gcsfuse --implicit-dirs waymo_open_dataset_motion_v_1_0_0 waymo_dataset
!pip3 install --upgrade pip
!pip3 install waymo-open-dataset-tf-2-6-0
!pip install keras==2.6.0

## **Necessary packages**

In [ ]:
import os
import math
import uuid
import itertools

import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np

import tensorflow as tf
from tensorflow.keras import Sequential
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Lambda, Reshape, BatchNormalization
from keras.layers import Conv1D, Dropout, Activation, MaxPool1D

from tensorflow.keras.optimizers import Adam
from keras.initializers import GlorotUniform

## **Global variables**

In [ ]:
NUM_USED_FILES = 1000
NUM_USED_FILES_VALID = 75

# Hyperparameters
BS = 128  #Batch size
BSV = 128 #Batch size for validation set

EPOCHS = 15
START_LR = 0.01
START_LR_TRAJ = 0.01
DROPOUT_RATE_SPATIAL = 0.4

# The same file to be used as visualization of the prediction as a proof
FILENAME = '/content/waymo_dataset/uncompressed/tf_example/training/training_tfexample.tfrecord-00000-of-01000'    
VALIDATION_FILENAME = '/content/waymo_dataset/uncompressed/tf_example/validation/validation_tfexample.tfrecord-00000-of-00150'    

STEP_TO_VISUALIZE = 0
EPOCH_TO_VISUALIZE = 2 # Chosen randomly

LATENT_DIM = 256  # The dimensions of the latent variables (and c in the LSTM model)

## **Get dataset**

In [ ]:
roadgraph_features = {
    "roadgraph_samples/dir": tf.io.FixedLenFeature(
        [20000, 3], tf.float32, default_value=None
    ),
    "roadgraph_samples/id": tf.io.FixedLenFeature(
        [20000, 1], tf.int64, default_value=None
    ),
    "roadgraph_samples/type": tf.io.FixedLenFeature(
        [20000, 1], tf.int64, default_value=None
    ),
    "roadgraph_samples/valid": tf.io.FixedLenFeature(
        [20000, 1], tf.int64, default_value=None
    ),
    "roadgraph_samples/xyz": tf.io.FixedLenFeature(
        [20000, 3], tf.float32, default_value=None
    ),
}

# Features of other agents.
state_features = {
    "state/id": tf.io.FixedLenFeature([128], tf.float32, default_value=None),
    "state/type": tf.io.FixedLenFeature([128], tf.float32, default_value=None),
    "state/is_sdc": tf.io.FixedLenFeature([128], tf.int64, default_value=None),
    "state/tracks_to_predict": tf.io.FixedLenFeature(
        [128], tf.int64, default_value=None
    ),
    "state/current/bbox_yaw": tf.io.FixedLenFeature(
        [128, 1], tf.float32, default_value=None
    ),
    "state/current/height": tf.io.FixedLenFeature(
        [128, 1], tf.float32, default_value=None
    ),
    "state/current/length": tf.io.FixedLenFeature(
        [128, 1], tf.float32, default_value=None
    ),
    "state/current/timestamp_micros": tf.io.FixedLenFeature(
        [128, 1], tf.int64, default_value=None
    ),
    "state/current/valid": tf.io.FixedLenFeature(
        [128, 1], tf.int64, default_value=None
    ),
    "state/current/vel_yaw": tf.io.FixedLenFeature(
        [128, 1], tf.float32, default_value=None
    ),
    "state/current/velocity_x": tf.io.FixedLenFeature(
        [128, 1], tf.float32, default_value=None
    ),
    "state/current/velocity_y": tf.io.FixedLenFeature(
        [128, 1], tf.float32, default_value=None
    ),
    "state/current/speed": tf.io.FixedLenFeature(
        [128, 1], tf.float32, default_value=None
    ),
    "state/current/width": tf.io.FixedLenFeature(
        [128, 1], tf.float32, default_value=None
    ),
    "state/current/x": tf.io.FixedLenFeature([128, 1], tf.float32, default_value=None),
    "state/current/y": tf.io.FixedLenFeature([128, 1], tf.float32, default_value=None),
    "state/current/z": tf.io.FixedLenFeature([128, 1], tf.float32, default_value=None),
    "state/future/bbox_yaw": tf.io.FixedLenFeature(
        [128, 80], tf.float32, default_value=None
    ),
    "state/future/height": tf.io.FixedLenFeature(
        [128, 80], tf.float32, default_value=None
    ),
    "state/future/length": tf.io.FixedLenFeature(
        [128, 80], tf.float32, default_value=None
    ),
    "state/future/timestamp_micros": tf.io.FixedLenFeature(
        [128, 80], tf.int64, default_value=None
    ),
    "state/future/valid": tf.io.FixedLenFeature(
        [128, 80], tf.int64, default_value=None
    ),
    "state/future/vel_yaw": tf.io.FixedLenFeature(
        [128, 80], tf.float32, default_value=None
    ),
    "state/future/velocity_x": tf.io.FixedLenFeature(
        [128, 80], tf.float32, default_value=None
    ),
    "state/future/velocity_y": tf.io.FixedLenFeature(
        [128, 80], tf.float32, default_value=None
    ),
    "state/future/width": tf.io.FixedLenFeature(
        [128, 80], tf.float32, default_value=None
    ),
    "state/future/x": tf.io.FixedLenFeature([128, 80], tf.float32, default_value=None),
    "state/future/y": tf.io.FixedLenFeature([128, 80], tf.float32, default_value=None),
    "state/future/z": tf.io.FixedLenFeature([128, 80], tf.float32, default_value=None),
    "state/past/bbox_yaw": tf.io.FixedLenFeature(
        [128, 10], tf.float32, default_value=None
    ),
    "state/past/height": tf.io.FixedLenFeature(
        [128, 10], tf.float32, default_value=None
    ),
    "state/past/length": tf.io.FixedLenFeature(
        [128, 10], tf.float32, default_value=None
    ),
    "state/past/timestamp_micros": tf.io.FixedLenFeature(
        [128, 10], tf.int64, default_value=None
    ),
    "state/past/valid": tf.io.FixedLenFeature([128, 10], tf.int64, default_value=None),
    "state/past/vel_yaw": tf.io.FixedLenFeature(
        [128, 10], tf.float32, default_value=None
    ),
    "state/past/velocity_x": tf.io.FixedLenFeature(
        [128, 10], tf.float32, default_value=None
    ),
    "state/past/velocity_y": tf.io.FixedLenFeature(
        [128, 10], tf.float32, default_value=None
    ),
    "state/past/speed": tf.io.FixedLenFeature(
        [128, 10], tf.float32, default_value=None
    ),
    "state/past/width": tf.io.FixedLenFeature(
        [128, 10], tf.float32, default_value=None
    ),
    "state/past/x": tf.io.FixedLenFeature([128, 10], tf.float32, default_value=None),
    "state/past/y": tf.io.FixedLenFeature([128, 10], tf.float32, default_value=None),
    "state/past/z": tf.io.FixedLenFeature([128, 10], tf.float32, default_value=None),
    "scenario/id": tf.io.FixedLenFeature([1], tf.string, default_value=None),
}

traffic_light_features = {
    "traffic_light_state/current/state": tf.io.FixedLenFeature(
        [1, 16], tf.int64, default_value=None
    ),
    "traffic_light_state/current/valid": tf.io.FixedLenFeature(
        [1, 16], tf.int64, default_value=None
    ),
    "traffic_light_state/current/id": tf.io.FixedLenFeature(
        [1, 16], tf.int64, default_value=None
    ),
    "traffic_light_state/current/x": tf.io.FixedLenFeature(
        [1, 16], tf.float32, default_value=None
    ),
    "traffic_light_state/current/y": tf.io.FixedLenFeature(
        [1, 16], tf.float32, default_value=None
    ),
    "traffic_light_state/current/z": tf.io.FixedLenFeature(
        [1, 16], tf.float32, default_value=None
    ),
    "traffic_light_state/past/state": tf.io.FixedLenFeature(
        [10, 16], tf.int64, default_value=None
    ),
    "traffic_light_state/past/valid": tf.io.FixedLenFeature(
        [10, 16], tf.int64, default_value=None
    ),
    # "traffic_light_state/past/id":
    # tf.io.FixedLenFeature([1, 16], tf.int64, default_value=None),
    "traffic_light_state/past/x": tf.io.FixedLenFeature(
        [10, 16], tf.float32, default_value=None
    ),
    "traffic_light_state/past/y": tf.io.FixedLenFeature(
        [10, 16], tf.float32, default_value=None
    ),
    "traffic_light_state/past/z": tf.io.FixedLenFeature(
        [10, 16], tf.float32, default_value=None
    ),
}

features_description = {}
features_description.update(roadgraph_features)
features_description.update(state_features)
features_description.update(traffic_light_features)


In [ ]:
def _parse_model_ready(value):
  """
  This function parses 1 scenario (where we max can have 128 different objects) out of many in 1 .tf file 
  into its basic elements. It returns all the necessary information about the scenario such as velocity 
  and position of each object for all the time points, which of the objects do currently or have/will exist
  decoded_example = tf.io.parse_single_example(value, features_description)

  Inputs:
    value: 1 tf file that is going to be parsed
  
  Returns:
    inputs: a dictionary that contains all the necessary information to do prediction about the future. 
  """
  decoded_example = tf.io.parse_single_example(value, features_description)


  #past_states includes all relevant information about pasts states that does not need to be normalized
  past_states = tf.stack([
      #Only includes valid values for each of the features by masking with valid vector. Currently sets all invalid enties to zero, but can easily be changed
      tf.where(decoded_example['state/past/valid'] > 0, decoded_example['state/past/speed'], tf.zeros_like(decoded_example['state/past/speed'])),
      tf.where(decoded_example['state/past/valid'] > 0, decoded_example['state/past/bbox_yaw'], tf.zeros_like(decoded_example['state/past/bbox_yaw'])),
      tf.where(decoded_example['state/past/valid'] > 0, decoded_example['state/past/velocity_x'], tf.zeros_like(decoded_example['state/past/velocity_x'])),
      tf.where(decoded_example['state/past/valid'] > 0, decoded_example['state/past/velocity_y'], tf.zeros_like(decoded_example['state/past/velocity_y'])),
      tf.where(decoded_example['state/past/valid'] > 0, decoded_example['state/past/vel_yaw'], tf.zeros_like(decoded_example['state/past/vel_yaw']))
  ], 1)  # shape (128,5,10) --> 128 = #of objects in the scenario, 5 properties (speed,bbox_yaw, ... vel_yaw), 10 steps in the past
  
  #past_xyz includes all relevant information about past states that needs to be normalized
  past_xyz = tf.stack([
      decoded_example['state/past/x'], 
      decoded_example['state/past/y'],
      decoded_example['state/past/z'],
  ], 1) # shape (128,3,10)


  #Same as for pasts_states but for current time point, see comment above
  cur_states = tf.stack([
      #Only includes valid values for each of the features by masking with valid vector. Currently sets all invalid entries to zero, but can easily be changed
      tf.where(decoded_example['state/current/valid'] > 0, decoded_example['state/current/speed'], tf.zeros_like(decoded_example['state/current/speed'])),
      tf.where(decoded_example['state/current/valid'] > 0, decoded_example['state/current/bbox_yaw'], tf.zeros_like(decoded_example['state/current/bbox_yaw'])),
      tf.where(decoded_example['state/current/valid'] > 0, decoded_example['state/current/velocity_x'], tf.zeros_like(decoded_example['state/current/velocity_x'])),
      tf.where(decoded_example['state/current/valid'] > 0, decoded_example['state/current/velocity_y'], tf.zeros_like(decoded_example['state/current/velocity_y'])),
      tf.where(decoded_example['state/current/valid'] > 0, decoded_example['state/current/vel_yaw'], tf.zeros_like(decoded_example['state/current/vel_yaw']))
  ], 1) # shape (128,5,1)

  cur_xyz = tf.stack([
      decoded_example['state/current/x'], 
      decoded_example['state/current/y'],
      decoded_example['state/current/z'],              
  ], 1) # shape (128,3,1)
  
  #Same as for pasts_states Note, future states are currently not being used and might be able to be removed.
  future_states = tf.stack([
        # If we use future states speed should probably be included but currently throws an error.
        #decoded_example['state/future/speed'],
        decoded_example['state/future/bbox_yaw'],
        decoded_example['state/future/velocity_x'],
        decoded_example['state/future/velocity_y'],
        decoded_example['state/future/vel_yaw']
    ], 1) # Will have shape (128,5,80)
  
  #Same as for past_xyz but for future time points
  future_xyz = tf.stack([
        decoded_example['state/future/x'], 
        decoded_example['state/future/y'],
        decoded_example['state/future/z'],                 
  ], 1) # Will have shape (128,3,80)

  past_is_valid = decoded_example['state/past/valid'] > 0
  current_is_valid = decoded_example['state/current/valid'] > 0
  future_is_valid = decoded_example['state/future/valid'] > 0
  is_valid = tf.concat([past_is_valid, current_is_valid], axis = 1)

  gt_future_is_valid = tf.concat([past_is_valid, current_is_valid, future_is_valid], 1)

  # If a sample was not seen at all in the past, we declare the sample as invalid.
  sample_is_valid = tf.reduce_any(tf.concat([past_is_valid, current_is_valid], 1), 1)

  # Gets the valid object types and set all invalid object types to 0.
  object_type = tf.where(sample_is_valid,decoded_example['state/type'], tf.zeros_like(decoded_example['state/type']))

  # Get all valid roadgraph types and roadgraph directions and setting all others to 0.
  roadgraph_type = tf.where(decoded_example['roadgraph_samples/valid'] > 0,decoded_example['roadgraph_samples/type'], tf.zeros_like(decoded_example['roadgraph_samples/type']))
  roadgraph_mask = tf.concat([decoded_example['roadgraph_samples/valid'] > 0, decoded_example['roadgraph_samples/valid'] > 0, decoded_example['roadgraph_samples/valid'] > 0],1)
  roadgraph_dir = tf.where(roadgraph_mask, decoded_example['roadgraph_samples/dir'], tf.zeros_like(decoded_example['roadgraph_samples/dir']))
    
  inputs = {
      'past_states': past_states,
      'past_xyz': past_xyz,
      'past_valid': past_is_valid,

      'current_states': cur_states,
      'current_xyz': cur_xyz,
      'current_valid': decoded_example['state/current/valid'] > 0,
      
      'future_states': future_states,
      'future_xyz': future_xyz,
      'future_valid': decoded_example['state/future/valid'] > 0,
      
      'object_type': decoded_example['state/type'],
      'is_valid': is_valid,
      'tracks_to_predict': decoded_example['state/tracks_to_predict'] > 0,
      'sample_is_valid': sample_is_valid,

      'roadgraph_samples/dir' :  roadgraph_dir,
      'roadgraph_samples/id' :  decoded_example['roadgraph_samples/id'],
      'roadgraph_samples/type' :  roadgraph_type,
      'roadgraph_samples/valid' :  decoded_example['roadgraph_samples/valid'],
      'roadgraph_samples/xyz' :  decoded_example['roadgraph_samples/xyz'],
      
      # For finding the gt in training and validation set
      'final_x': tf.reshape(decoded_example['state/future/x'][:,-1], (-1,1)), 
      'final_y': tf.reshape(decoded_example['state/future/y'][:,-1], (-1,1))
  }
  return inputs  

In [ ]:
def get_dataset(path, num_used_files, bs, alt_cizgi = 0):
    data_files = os.listdir(path)
    data_files = data_files[alt_cizgi: alt_cizgi + num_used_files]          
    dataset_plot = tf.data.TFRecordDataset([os.path.join(path, f) for f in data_files], num_parallel_reads=4)
    dataset = dataset_plot.map(_parse_model_ready)     
    dataset = dataset.batch(bs)                   
    return dataset, dataset_plot

# **Construct the training set on a normalized map**

In [ ]:
def const_train_set(inputs, gt_func):

    #Normalizes the input that needs to be normalized
    normalized_inputs = normalize(inputs)

    bs, num_agents, num_features, past_time_steps = normalized_inputs['past_states'].shape
    time_steps = past_time_steps + 1 
    bs, road_num, _ = inputs['roadgraph_samples/id'].shape
    
    # Adds past and current states to input vector
    X_past = tf.reshape(normalized_inputs['past_states'], [bs, num_agents * num_features, past_time_steps])
    X_current = tf.reshape(normalized_inputs['current_states'], [bs, num_agents * num_features, 1])
    X_time = tf.concat([X_past, X_current], axis = 2)

    # Adds object types to input vector
    A = tf.broadcast_to(tf.expand_dims(inputs['object_type'], axis = 2), shape=[bs, num_agents, time_steps])
    X_time = tf.concat([X_time, A], axis=1)

    A = tf.broadcast_to(tf.expand_dims(tf.cast(inputs['tracks_to_predict'], dtype = 'float32'), axis = 2), shape=[bs, num_agents, time_steps])
    X_time = tf.concat([X_time, A], axis=1)

    # Adds roadgraph types (e.g. LaneCenter-Freeway, Roadline-SolidSingleYellow)
    X_spatial = tf.cast(inputs['roadgraph_samples/type'], dtype = 'float32')

    # Adds the direction of the roadgraph
    A = tf.cast(tf.reshape(inputs['roadgraph_samples/dir'][:,:,0], (bs, 20000,1)), dtype = 'float32')
    X_spatial = tf.concat([X_spatial, A], axis=1)
    A = tf.cast(tf.reshape(inputs['roadgraph_samples/dir'][:,:,1], (bs, 20000,1)), dtype = 'float32')
    X_spatial = tf.concat([X_spatial, A], axis=1)
    A = tf.cast(tf.reshape(inputs['roadgraph_samples/dir'][:,:,2], (bs, 20000,1)), dtype = 'float32')
    X_spatial = tf.concat([X_spatial, A], axis=1)

    Y = gt_func(inputs, normalized_inputs)

    return X_time, X_spatial, Y, normalized_inputs['center_x'], normalized_inputs['center_y']

In [ ]:
def normalize(inputs):
  # This function moves all scense to origin
  
  # Finds the center values for each scene in the batch. shape of center_? = (128,)
  center_x, center_y, center_z = find_center(inputs['roadgraph_samples/xyz'], inputs['roadgraph_samples/valid'])
  
  # Normalizes the roadgraph by subtracting the center value.
  # The transposes are there to make the broadcasting work properly in the correct dimension.
  # Also extracts the valid values and sets all other values to zero.
  bs = inputs['roadgraph_samples/xyz'].shape[0]
  roadgraph_x_norm = tf.transpose(tf.transpose(inputs['roadgraph_samples/xyz'][:,:,0]) - center_x)
  roadgraph_x_norm_valid = tf.where(tf.reshape(inputs['roadgraph_samples/valid']>0, (bs,-1)), roadgraph_x_norm, tf.zeros_like(roadgraph_x_norm))

  roadgraph_y_norm = tf.transpose(tf.transpose(inputs['roadgraph_samples/xyz'][:,:,1]) - center_x)
  roadgraph_y_norm_valid = tf.where(tf.reshape(inputs['roadgraph_samples/valid']>0, (bs,-1)), roadgraph_y_norm, tf.zeros_like(roadgraph_y_norm))
  
  roadgraph_z_norm = tf.transpose(tf.transpose(inputs['roadgraph_samples/xyz'][:,:,2]) - center_x)
  roadgraph_z_norm_valid = tf.where(tf.reshape(inputs['roadgraph_samples/valid']>0, (bs,-1)), roadgraph_z_norm, tf.zeros_like(roadgraph_z_norm))
  

  # Normalizes the past coordinates by subtracting the center value.
  # The transposes are there to make the broadcasting work properly in the correct dimension.
  # Also extracts the valid values and sets all other values to zero.
  bs, num_agents, _, past_timesteps = inputs['past_xyz'].shape
  past_states_x_norm = inputs['past_xyz'][:,:,0,:] - tf.transpose(tf.broadcast_to(tf.broadcast_to(center_x,(num_agents,bs)), (past_timesteps,num_agents,bs)), (2,1,0))
  past_states_x_norm_valid = tf.where(inputs['past_valid'], past_states_x_norm, tf.zeros_like(past_states_x_norm))
  past_states_y_norm = inputs['past_xyz'][:,:,1,:] - tf.transpose(tf.broadcast_to(tf.broadcast_to(center_y,(num_agents,bs)), (past_timesteps,num_agents,bs)), (2,1,0))
  past_states_y_norm_valid = tf.where(inputs['past_valid'], past_states_y_norm, tf.zeros_like(past_states_y_norm))
  past_states_z_norm = inputs['past_xyz'][:,:,2,:] - tf.transpose(tf.broadcast_to(tf.broadcast_to(center_z,(num_agents,bs)), (past_timesteps,num_agents,bs)), (2,1,0))
  past_states_z_norm_valid = tf.where(inputs['past_valid'], past_states_z_norm, tf.zeros_like(past_states_z_norm))

  past_xyz = tf.stack([past_states_x_norm_valid, past_states_y_norm_valid, past_states_z_norm_valid], 2)
  past_states =  tf.concat([past_xyz, inputs['past_states']], 2)


  # Normalizes the current coordinates by subtracting the center value.
  # The transposes are there to make the broadcasting work properly in the correct dimension.
  # Also extracts the valid values and sets all other values to zero.
  bs, num_agents, _, current_timesteps = inputs['current_xyz'].shape
  curr_states_x_norm = inputs['current_xyz'][:,:,0,:] - tf.transpose(tf.broadcast_to(tf.broadcast_to(center_x,(num_agents,bs)), (current_timesteps,num_agents,bs)), (2,1,0))
  curr_states_x_norm_valid = tf.where(inputs['current_valid'], curr_states_x_norm, tf.zeros_like(curr_states_x_norm))
  curr_states_y_norm = inputs['current_xyz'][:,:,1,:] - tf.transpose(tf.broadcast_to(tf.broadcast_to(center_y,(num_agents,bs)), (current_timesteps,num_agents,bs)), (2,1,0))
  curr_states_y_norm_valid = tf.where(inputs['current_valid'], curr_states_y_norm, tf.zeros_like(curr_states_y_norm))
  curr_states_z_norm = inputs['current_xyz'][:,:,2,:] - tf.transpose(tf.broadcast_to(tf.broadcast_to(center_z,(num_agents,bs)), (current_timesteps,num_agents,bs)), (2,1,0))
  curr_states_z_norm_valid = tf.where(inputs['current_valid'], curr_states_z_norm, tf.zeros_like(curr_states_z_norm))  
  
  curr_xyz = tf.stack([curr_states_x_norm_valid, curr_states_y_norm_valid, curr_states_z_norm_valid], 2)
  curr_states =  tf.concat([curr_xyz, inputs['current_states']], 2)

  # Normalizes the future coordinates by subtracting the center value.
  # The transposes are there to make the broadcasting work properly in the correct dimension.
  # Also extracts the valid values and sets all other values to zero.
  bs, num_agents, _, future_timesteps = inputs['future_xyz'].shape
  future_states_x_norm = inputs['future_xyz'][:,:,0,:] - tf.transpose(tf.broadcast_to(tf.broadcast_to(center_x,(num_agents,bs)), (future_timesteps,num_agents,bs)), (2,1,0))
  future_states_x_norm_valid = tf.where(inputs['future_valid'], future_states_x_norm, tf.zeros_like(future_states_x_norm))
  future_states_y_norm = inputs['future_xyz'][:,:,1,:] - tf.transpose(tf.broadcast_to(tf.broadcast_to(center_y,(num_agents,bs)), (future_timesteps,num_agents,bs)), (2,1,0))
  future_states_y_norm_valid = tf.where(inputs['future_valid'], future_states_y_norm, tf.zeros_like(future_states_y_norm))
  future_states_z_norm = inputs['future_xyz'][:,:,2,:] - tf.transpose(tf.broadcast_to(tf.broadcast_to(center_z,(num_agents,bs)), (future_timesteps,num_agents,bs)), (2,1,0))
  future_states_z_norm_valid = tf.where(inputs['future_valid'], future_states_z_norm, tf.zeros_like(future_states_z_norm))  
  
  future_xyz = tf.stack([future_states_x_norm_valid, future_states_y_norm_valid, future_states_z_norm_valid], 2)
  future_states =  tf.concat([future_xyz, inputs['future_states']], 2)


  normalized_inputs = {
      'past_states': past_states,
      'current_states': curr_states,
      'future_states': future_states,
      'roadgraph_samples/x':  tf.reshape(roadgraph_x_norm_valid, (bs,-1,1)),
      'roadgraph_samples/y':  tf.reshape(roadgraph_y_norm_valid, (bs,-1,1)),
      'roadgraph_samples/z':  tf.reshape(roadgraph_z_norm_valid, (bs,-1,1)),

      # For finding the gt in training and validation set
      'final_x': tf.reshape(future_states_x_norm_valid[:,:,-1], (bs, num_agents, 1)),
      'final_y': tf.reshape(future_states_y_norm_valid[:,:,-1], (bs, num_agents, 1)),
      'center_x': center_x,
      'center_y': center_y
  }
  return normalized_inputs 

In [ ]:
def find_center(roadgraph_xyz, roadgraph_valid):
    #finds the center of the map by finding the largest and the smallest value of the roads, and average these values.
    
    #Finds the max for each scene in the batch by taking the max over the second dimension (the road values).
    #Invalid data is filled with large negative value in order to not be the maximum value and thus influence the result.
    #The same approach is used for finding the minimum value but with a large positive value.
    max_x = tf.reduce_max(tf.where(roadgraph_valid[:,:,0]>0, roadgraph_xyz[:,:,0], tf.ones_like(roadgraph_xyz[:,:,0])*-10000000), 1)
    min_x = tf.reduce_min(tf.where(roadgraph_valid[:,:,0]>0, roadgraph_xyz[:,:,0], tf.ones_like(roadgraph_xyz[:,:,0])*10000000), 1)
    
    max_y = tf.reduce_max(tf.where(roadgraph_valid[:,:,0]>0, roadgraph_xyz[:,:,1], tf.ones_like(roadgraph_xyz[:,:,1])*-10000000), 1)
    min_y = tf.reduce_min(tf.where(roadgraph_valid[:,:,0]>0, roadgraph_xyz[:,:,1], tf.ones_like(roadgraph_xyz[:,:,1])*10000000), 1)
    
    max_z = tf.reduce_max(tf.where(roadgraph_valid[:,:,0]>0, roadgraph_xyz[:,:,2], tf.ones_like(roadgraph_xyz[:,:,2])*-10000000), 1)
    min_z = tf.reduce_min(tf.where(roadgraph_valid[:,:,0]>0, roadgraph_xyz[:,:,2], tf.ones_like(roadgraph_xyz[:,:,2])*10000000), 1)

    center_x = (max_x + min_x)/2
    center_y = (max_y + min_y)/2
    center_z = (max_z + min_z)/2

    return center_x, center_y, center_z

# **Goal Prediction Part**

## **Construct the LSTM model for end point prediction**

In [ ]:
def GoalPredModel(num_states_steps, latent_dim, num_agents_to_predict, num_end_points):
      """Goal Prediction model
      param num_states_steps: 10 past + 1 current scenario
      param lated_dim: the dimension of the latent diemsions between the LSTM blocks
      param num_agents_to_predict: Number of agents that the system needs to predict
      param num_end_points: Number of end points the system needs to predict = 2 (x and y coordinate for 1 point)
      """

      #The length of the large input vector. Depends on how many features we include.
      input_vec_length = 1280 
      num_static_features = 20000*4
      time_inputs = Input(shape=(input_vec_length, num_states_steps))
      static_inputs = Input(shape=(num_static_features,1))
      a0 = Input(shape=(latent_dim,), name='a0')
      c0 = Input(shape=(latent_dim,), name='c0')
      a = a0
      c = c0

      # Create the encoder
      for t in range(num_states_steps):
          # Select the "t"th time step vector from enc_input. 
          x_time = Lambda(lambda z: z[:, :, t])(time_inputs)
          # Reshape x 
          x_time = Reshape((1, input_vec_length))(x_time)
          x_time = BatchNormalization()(x_time)                                 
          # LSTM_cell
          a, _, c = LSTM(latent_dim, kernel_initializer=GlorotUniform(), return_state = True)(x_time, initial_state=[a, c])

      #Convolve spatial features
      x_spatial = Conv1D(4, 5, strides = 2, padding = 'same', kernel_initializer=GlorotUniform())(static_inputs)
      x_spatial = MaxPool1D(pool_size=2, strides=2)(x_spatial)
      x_spatial = Conv1D(8, 5, strides = 2, padding = 'same', kernel_initializer=GlorotUniform())(x_spatial)
      x_spatial = MaxPool1D(pool_size=2, strides=2)(x_spatial)
      x_spatial = Conv1D(8, 5, strides = 2, padding = 'same', kernel_initializer=GlorotUniform())(x_spatial)
      x_spatial = MaxPool1D(pool_size=2, strides=2)(x_spatial)
      x_spatial = Conv1D(16, 5, strides = 2, padding = 'same', kernel_initializer=GlorotUniform())(x_spatial)
      x_spatial = MaxPool1D(pool_size=2, strides=2)(x_spatial)

      x_spatial = tf.keras.layers.Flatten()(x_spatial)
      x_spatial = Dropout(rate=DROPOUT_RATE_SPATIAL)(x_spatial)
      a = tf.keras.layers.Flatten()(a)

      x = tf.concat([a, x_spatial], axis = 1)

      # To hold the outputs
      outputs = []
      out = Dense(num_agents_to_predict*num_end_points, kernel_initializer=GlorotUniform())(x)
      outputs.append(out)

      model = Model(inputs=[time_inputs, static_inputs, a0, c0], outputs=outputs)
      return model

## **Get the ground truth data**

In [ ]:
def get_gt_final(inputs, normalized_inputs):
    """Returns a concatenated tensor of shape (bs,256,1), where 256=128*2 (for x and y coordinate)
    The only values that are non-zero in the return tensor are the ones that we actually are interested in.
    Observe the first 8 of the indicies represent the x and the last 8 represents the y coordinate
    """
    
    Yx = normalized_inputs['final_x']
    Yy = normalized_inputs['final_y']
    tracks = inputs['tracks_to_predict']

    Yx = tf.math.multiply(Yx, tf.expand_dims(tf.cast(tracks, dtype = 'float32'), axis = 2))
    Yx = Yx[:,0:8,:]

    Yy = tf.math.multiply(Yy, tf.expand_dims(tf.cast(tracks, dtype = 'float32'), axis = 2))
    Yy = Yy[:,0:8,:]
    
    Y = tf.concat([Yx, Yy], axis = 1)
    return Y

## **Visualization**

The functions in this subsection is from:

Scott Ettinger, Shuyang Cheng, Benjamin Caine, Chenxi Liu, Hang Zhao, Sabeek Pradhan, Yuning Chai, Ben Sapp,
Charles R. Qi, Yin Zhou, Zoey Yang, Aur’elien Chouard, Pei Sun, Jiquan Ngiam, Vijay Vasudevan, Alexander
McCauley, Jonathon Shlens, and Dragomir Anguelov. Large scale interactive motion forecasting for autonomous
driving: The waymo open motion dataset.
6

In [ ]:
def create_figure_and_axes(size_pixels):
  """Initializes a unique figure and axes for plotting."""
  fig, ax = plt.subplots(1, 1, num=uuid.uuid4())

  # Sets output image to pixel resolution.
  dpi = 100
  size_inches = size_pixels / dpi
  fig.set_size_inches([size_inches, size_inches])
  fig.set_dpi(dpi)
  fig.set_facecolor('white')
  ax.set_facecolor('white')
  ax.xaxis.label.set_color('black')
  ax.tick_params(axis='x', colors='black')
  ax.yaxis.label.set_color('black')
  ax.tick_params(axis='y', colors='black')
  fig.set_tight_layout(True)
  ax.grid(False)
  return fig, ax

In [ ]:
def fig_canvas_image(fig):
  """Returns a [H, W, 3] uint8 np.array image from fig.canvas.tostring_rgb()."""
  # Just enough margin in the figure to display xticks and yticks.
  fig.subplots_adjust(
      left=0.08, bottom=0.08, right=0.98, top=0.98, wspace=0.0, hspace=0.0)
  fig.canvas.draw()
  data = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
  return data.reshape(fig.canvas.get_width_height()[::-1] + (3,))

In [ ]:
def get_colormap(num_agents):
  """Compute a color map array of shape [num_agents, 4]."""
  colors = cm.get_cmap('jet', num_agents)
  colors = colors(range(num_agents))
  np.random.shuffle(colors)
  colors[0] = [0.,0.,0.5,1.,]
  colors[1] = [0.5,0.,0.,1.,]
  colors[2] = [0.,0.5,0.,1.,]

  colors[3] = [0.5,0.5,0.,1.,]
  colors[4] = [0.5,0.,0.5,1.,]
  colors[5] = [0.,0.5,0.5,1.,]

  colors[6] = [0.2,0.7,0.1,1.,]
  colors[7] = [0.5,0.5,0.5,1.,]
  return colors

In [ ]:
def get_viewport(all_states, all_states_mask):
  """Gets the region containing the data.

  Args:
    all_states: states of agents as an array of shape [num_agents, num_steps,
      2].
    all_states_mask: binary mask of shape [num_agents, num_steps] for
      `all_states`.

  Returns:
    center_y: float. y coordinate for center of data.
    center_x: float. x coordinate for center of data.
    width: float. Width of data.
  """
  valid_states = all_states[all_states_mask]
  all_y = valid_states[..., 1]
  all_x = valid_states[..., 0]

  center_y = (np.max(all_y) + np.min(all_y)) / 2
  center_x = (np.max(all_x) + np.min(all_x)) / 2

  range_y = np.ptp(all_y)
  range_x = np.ptp(all_x)

  width = max(range_y, range_x)

  return center_y, center_x, width

In [ ]:
def visualize_one_step(states,
                       mask,
                       roadgraph,
                       title,
                       center_y,
                       center_x,
                       width,
                       color_map,
                       size_pixels=1000):
  """Generate visualization for a single step."""

  # Create figure and axes.
  fig, ax = create_figure_and_axes(size_pixels=size_pixels)

  # Plot roadgraph.
  rg_pts = roadgraph[:, :2].T
  ax.plot(rg_pts[0, :], rg_pts[1, :], 'k.', alpha=1, ms=2)

  masked_x = states[:, 0][mask]
  masked_y = states[:, 1][mask]
  colors = color_map[mask]

  # Plot agent current position.
  ax.scatter(
      masked_x,
      masked_y,
      marker='o',
      linewidths=3,
      color=colors,
  )

  # Title.
  ax.set_title(title)

  # Set axes.  Should be at least 10m on a side and cover 160% of agents.
  size = max(10, width * 1.0)
  ax.axis([
      -size / 2 + center_x, size / 2 + center_x, -size / 2 + center_y,
      size / 2 + center_y
  ])
  ax.set_aspect('equal')

  image = fig_canvas_image(fig)
  plt.close(fig)
  return image

In [ ]:
def visualize_all_agents_smooth(decoded_example,
    size_pixels=1000,
):
  """Visualizes all agent predicted trajectories in a serie of images.

  Args:
    decoded_example: Dictionary containing agent info about all modeled agents.
    size_pixels: The size in pixels of the output image.

  Returns:
    T of [H, W, 3] uint8 np.arrays of the drawn matplotlib's figure canvas.
  """
  # [num_agents, num_past_steps, 2] float32.
  past_states = tf.stack(
      [decoded_example['state/past/x'], decoded_example['state/past/y']],
      -1).numpy()

  num_agents, num_past_steps, _ = past_states.shape

  past_states_mask = decoded_example['state/past/valid'].numpy() > 0.0

  # [num_agents, 1, 2] float32.
  current_states = tf.stack(
      [decoded_example['state/current/x'], decoded_example['state/current/y']],
      -1).numpy()
  current_states_mask = decoded_example['state/current/valid'].numpy() > 0.0

  # [num_agents, num_future_steps, 2] float32.
  future_states = tf.stack(
      [decoded_example['state/future/x'], decoded_example['state/future/y']],
      -1).numpy()
  num_future_steps = future_states.shape[1]
  future_states_mask = decoded_example['state/future/valid'].numpy() > 0.0
  
  # [num_points, 3] float32.
  roadgraph_xyz = decoded_example['roadgraph_samples/xyz'].numpy()

  color_map = get_colormap(num_agents)

  # [num_agens, num_past_steps + 1 + num_future_steps, depth] float32.
  all_states = np.concatenate([past_states, current_states, future_states], 1)

  # [num_agens, num_past_steps + 1 + num_future_steps] float32.
  all_states_mask = np.concatenate(
      [past_states_mask, current_states_mask, future_states_mask], 1)
  
  center_y, center_x, width = get_viewport(all_states, all_states_mask)

  images = []

  # Generate images from past time steps.
  for i, (s, m) in enumerate(
      zip(
          np.split(past_states, num_past_steps, 1),
          np.split(past_states_mask, num_past_steps, 1))):
    im = visualize_one_step(s[:, 0], m[:, 0], roadgraph_xyz,
                            'past: %d' % (num_past_steps - i), center_y,
                            center_x, width, color_map, size_pixels)
    images.append(im)

  # Generate one image for the current time step.
  s = current_states
  m = current_states_mask

  im = visualize_one_step(s[:, 0], m[:, 0], roadgraph_xyz, 'current', center_y,
                          center_x, width, color_map, size_pixels)
  images.append(im)

  predict_agent_mask = decoded_example['state/tracks_to_predict'].numpy() > 0.0
  predict_agent_mask = tf.broadcast_to(tf.expand_dims(predict_agent_mask, axis=1), [num_agents, num_future_steps])
  future_states_mask = np.logical_and(future_states_mask, predict_agent_mask)
  print(future_states_mask[0:10,-1])
  
  # Generate images from future time steps.
  for i, (s, m) in enumerate(
      zip(
          np.split(future_states, num_future_steps, 1),
          np.split(future_states_mask, num_future_steps, 1))):
    im = visualize_one_step(s[:, 0], m[:, 0], roadgraph_xyz,
                            'future: %d' % (i + 1), center_y, center_x, width,
                            color_map, size_pixels)
    images.append(im)

  return images

In [ ]:
def visualize_all_agents_smooth_w_np(decoded_example, prediction_file_x, prediction_file_y,
    size_pixels=1000,):
  """Visualizes all agent predicted trajectories in a serie of images.

  Args:
    decoded_example: Dictionary containing agent info about all modeled agents.
    size_pixels: The size in pixels of the output image.

  Returns:
    T of [H, W, 3] uint8 np.arrays of the drawn matplotlib's figure canvas.
  """
  # [num_agents, num_past_steps, 2] float32.
  past_states = tf.stack(
      [decoded_example['state/past/x'], decoded_example['state/past/y']],
      -1).numpy()

  num_agents, num_past_steps, _ = past_states.shape

  past_states_mask = decoded_example['state/past/valid'].numpy() > 0.0

  # [num_agents, 1, 2] float32.
  current_states = tf.stack(
      [decoded_example['state/current/x'], decoded_example['state/current/y']],
      -1).numpy()
  current_states_mask = decoded_example['state/current/valid'].numpy() > 0.0

  # [num_agents, num_future_steps, 2] float32.
  future_states = np.stack(
      [prediction_file_x, prediction_file_y],
      -1)
  num_future_steps = future_states.shape[1]
  future_states_mask = decoded_example['state/future/valid'].numpy() > 0.0
  
  # [num_points, 3] float32.
  roadgraph_xyz = decoded_example['roadgraph_samples/xyz'].numpy()

  color_map = get_colormap(num_agents)

  # [num_agens, num_past_steps + 1 + num_future_steps, depth] float32.
  all_states = np.concatenate([past_states, current_states, future_states], 1)

  # [num_agens, num_past_steps + 1 + num_future_steps] float32.
  all_states_mask = np.concatenate(
      [past_states_mask, current_states_mask, future_states_mask], 1)
  
  center_y, center_x, width = get_viewport(all_states, all_states_mask)

  images = []

  # Generate images from past time steps.
  for i, (s, m) in enumerate(
      zip(
          np.split(past_states, num_past_steps, 1),
          np.split(past_states_mask, num_past_steps, 1))):
    im = visualize_one_step(s[:, 0], m[:, 0], roadgraph_xyz,
                            'past: %d' % (num_past_steps - i), center_y,
                            center_x, width, color_map, size_pixels)
    images.append(im)

  # Generate one image for the current time step.
  s = current_states
  m = current_states_mask

  im = visualize_one_step(s[:, 0], m[:, 0], roadgraph_xyz, 'current', center_y,
                          center_x, width, color_map, size_pixels)
  images.append(im)

  predict_agent_mask = decoded_example['state/tracks_to_predict'].numpy() > 0.0
  predict_agent_mask = tf.broadcast_to(tf.expand_dims(predict_agent_mask, axis=1), [num_agents, num_future_steps])
  future_states_mask = np.logical_and(future_states_mask, predict_agent_mask)
  #print(future_states_mask[0:10,-1])
  
  # Generate images from future time steps.
  for i, (s, m) in enumerate(
      zip(
          np.split(future_states, num_future_steps, 1),
          np.split(future_states_mask, num_future_steps, 1))):
    im = visualize_one_step(s[:, 0], m[:, 0], roadgraph_xyz,
                            'future: %d' % (i + 1), center_y, center_x, width,
                            color_map, size_pixels)
    images.append(im)

  return images

## **Plot**

In [ ]:
def calculate_plots(visualization_file, predicted_values_x, predicted_values_y, gt_values_x, gt_values_y):
    # Plot the end point for the agents we are interested in. First for the GT case (not the predicted one).
    #images = visualize_all_agents_smooth(visualization_file)
    #plt.rcParams['figure.figsize'] = [10, 10]
    #plt.imshow(images[-1])
    #plt.show()

    # Plot the end point for the agents we are interested in. Second for the predicted one.
    gt_file_x = visualization_file['state/future/x'].numpy()
    gt_file_x[:,-1][0:8] = gt_values_x 
    gt_file_y = visualization_file['state/future/y'].numpy()
    gt_file_y[:,-1][0:8] = gt_values_y

    images = visualize_all_agents_smooth_w_np(visualization_file, gt_file_x, gt_file_y)
    plt.rcParams['figure.figsize'] = [10, 10]
    plt.imshow(images[-1])
    plt.show()

    # Plot the end point for the agents we are interested in. Second for the predicted one.
    prediction_file_x = visualization_file['state/future/x'].numpy()
    prediction_file_x[:,-1][0:8] = predicted_values_x 
    prediction_file_y = visualization_file['state/future/y'].numpy()
    prediction_file_y[:,-1][0:8] = predicted_values_y

    images = visualize_all_agents_smooth_w_np(visualization_file, prediction_file_x, prediction_file_y)
    plt.rcParams['figure.figsize'] = [10, 10]
    plt.imshow(images[-1])
    plt.show()

## **Helper functions for training the goal prediction model**

In [ ]:
def train_1_epoch(epoch, train_dataset):
    # Set the batch losses to be an empty list to only contain losses from this epochs batches.
    logits_visualize, center_x_visualize, center_y_visualize, y_gt_visualize = 0, 0, 0, 0
    train_batch_losses = []
    
    # Iterate over the batches of the dataset.
    for step, batch in enumerate(train_dataset):
        # Send in 1 batch to the function train_step batch.
        # 'batch' is a DICTIONARY containing the past position, velocity, current position,
        # velocity etc. for all the objects in the scenario for all the time indices for
        # all batches. 
        train_batch_time, train_batch_static, y_batch_train, center_x, center_y = const_train_set(batch, get_gt_final)
        bs = train_batch_time.shape[0]
        a0 = np.zeros((bs, LATENT_DIM))
        c0 = np.zeros((bs, LATENT_DIM))

        with tf.GradientTape() as tape:
            # Run the forward pass
            logits = model([train_batch_time, train_batch_static ,a0,c0], training=True)
            y_batch_train = tf.reshape(y_batch_train, (y_batch_train.shape[0:2]))

            # Saves gt and predicted values to be able to visualize the results 
            if step == STEP_TO_VISUALIZE and epoch % EPOCH_TO_VISUALIZE == 0:# and epoch != 0:
              y_gt_visualize = y_batch_train
              logits_visualize = logits
              center_x_visualize = center_x
              center_y_visualize = center_y
            
            # Compute the loss value
            weights = batch['tracks_to_predict'][:,:8]
            weights = tf.concat([weights, weights], axis = 1)

            valid_logits = tf.boolean_mask(logits, weights)
            valid_gt = tf.boolean_mask(y_batch_train, weights)
            loss_value = loss_fn(valid_gt, valid_logits)

        # Use the gradient tape to automatically retrieve
        # the gradients of the trainable variables with respect to the loss.
        grads = tape.gradient(loss_value, model.trainable_weights)

        # Run one step of gradient descent by updating
        # the value of the variables to minimize the loss
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        train_batch_losses.append(loss_value)
    # Prints the training loss after each epoch
    print("The mean training loss after %d epochs: %.4f"% (epoch, float(sum(train_batch_losses)/len(train_batch_losses))))
    train_epoch_losses.append(float(sum(train_batch_losses)/len(train_batch_losses)))
    return logits_visualize, center_x_visualize, center_y_visualize, y_gt_visualize

In [ ]:
def calc_val_loss(valid_dataset): 
  valid_batch_losses = [] 
  logits_visualize_valid, center_x_valid_visualize, center_y_valid_visualize, y_gt_visualize_valid = 0,0,0,0
  for step, batch in enumerate(valid_dataset):
    valid_time_batch, valid_static_batch,y_batch_valid, center_x_valid, center_y_valid = const_train_set(batch, get_gt_final)
    bs = valid_time_batch.shape[0]
    a0 = np.zeros((bs, LATENT_DIM))
    c0 = np.zeros((bs, LATENT_DIM))

    with tf.GradientTape() as tape:
      # Run the forward pass of the layer.
      logits = model([valid_time_batch,valid_static_batch,a0,c0], training=False)
      y_batch_valid = tf.reshape(y_batch_valid, (y_batch_valid.shape[0:2]))

      # Saves gt and predicted values to be able to visualize the results 
      if step == STEP_TO_VISUALIZE and epoch % EPOCH_TO_VISUALIZE == 0 and epoch != 0:
        y_gt_visualize_valid = y_batch_valid
        logits_visualize_valid = logits
        center_x_valid_visualize = center_x_valid
        center_y_valid_visualize = center_y_valid
      
      # Compute the loss value for this minibatch.
      loss_value = loss_fn(y_batch_valid, logits)

    valid_batch_losses.append(loss_value)

  print("The mean validation loss after %d epochs: %.4f"% (epoch, float(sum(valid_batch_losses)/len(valid_batch_losses))))
  valid_epoch_losses.append(float(sum(valid_batch_losses)/len(valid_batch_losses)))

  return logits_visualize_valid, center_x_valid_visualize, center_y_valid_visualize, y_gt_visualize_valid

In [ ]:
model = GoalPredModel(11, LATENT_DIM, 8, 2)

lr = START_LR
optimizer = Adam(learning_rate=lr, beta_1=0.9, beta_2=0.999)
loss_fn = tf.keras.losses.MeanSquaredError()

train_epoch_losses = []
valid_epoch_losses = []

In [ ]:
dataset_for_visualization = tf.data.TFRecordDataset(FILENAME, compression_type='')    # parse the tf record
dataset_for_visualization_valid = tf.data.TFRecordDataset(VALIDATION_FILENAME, compression_type='')    # parse the tf record


for step, batch in enumerate(dataset_for_visualization.batch(BS)):
    if step == STEP_TO_VISUALIZE:
        for example in batch:
            visualization_file = tf.io.parse_single_example(example, features_description)
            break

for step, batch in enumerate(dataset_for_visualization_valid.batch(BSV)):
    if step == STEP_TO_VISUALIZE:
        for example in batch:
            visualization_file_valid = tf.io.parse_single_example(example, features_description)
            break

visualization_file_for_prediction = dataset_for_visualization.map(_parse_model_ready)
visualization_file_for_prediction_valid = dataset_for_visualization_valid.map(_parse_model_ready)

In [ ]:
#Get the train sets
train_path = '/content/waymo_dataset/uncompressed/tf_example/training/'
valid_path = '/content/waymo_dataset/uncompressed/tf_example/validation/'
train_dataset, train_dataset_plot = get_dataset(train_path, NUM_USED_FILES, BS)
valid_dataset, _ = get_dataset(valid_path, NUM_USED_FILES_VALID, BSV)
test_dataset, _ = get_dataset(valid_path, NUM_USED_FILES_VALID, BSV, alt_cizgi = 75)

## **Train**

In [ ]:
for epoch in range(EPOCHS):
    print('\nStart of epoch %d' % (epoch,))
    
    # Run 1 epoch on the training dataset and update model parameters
    logits_visualize, center_x_visualize, center_y_visualize, y_gt_visualize = train_1_epoch(epoch, train_dataset)

    # Calculate and print the validation loss
    logits_visualize_valid, center_x_valid_visualize, center_y_valid_visualize, y_gt_visualize_valid = calc_val_loss(valid_dataset)
        
    # Visualize the results
    if epoch % EPOCH_TO_VISUALIZE == 0 and epoch != 0:
        calculate_plots(visualization_file, logits_visualize[0][0:8]+center_x_visualize[0], logits_visualize[0][8:]+center_y_visualize[0], y_gt_visualize[0][0:8]+center_x_visualize[0], y_gt_visualize[0][8:]+center_y_visualize[0])  #adding the center values to the logits to make the vizualisation in the original coordinate system
        calculate_plots(visualization_file_valid, logits_visualize_valid[0][0:8]+center_x_valid_visualize[0], logits_visualize_valid[0][8:]+center_y_valid_visualize[0], y_gt_visualize_valid[0][0:8]+center_x_valid_visualize[0], y_gt_visualize_valid[0][8:]+center_y_valid_visualize[0])  #adding the center values to the logits to make the vizualisation in the original coordinate system

In [ ]:
# Show the results
plt.plot(np.arange(0, len(train_epoch_losses)), train_epoch_losses)
plt.plot(np.arange(0, len(valid_epoch_losses)), valid_epoch_losses)
plt.show()

# **Trajectory Prediction Part**

## **Calculate the GT for future steps**

In [ ]:
def get_trajectory_gt(inputs, normalized_inputs):
    """Returns a concatenated tensor of shape (bs,1280,1), where 1280=8*2*80 (for x and y coordinate)
    The only values that are non-zero in the return tensor are the ones that we actually are interested in.
    """
    
    Yx = normalized_inputs['future_states'][:,:,0,:]   # is of shape (bs, #agents, time steps in future (80)) -- x coordinate
    Yy = normalized_inputs['future_states'][:,:,1,:]   # is of shape (bs, #agents, time steps in future (80)) -- y coordinate
    tracks = inputs['tracks_to_predict']

    Yx = tf.math.multiply(Yx, tf.expand_dims(tf.cast(tracks, dtype = 'float32'), axis = 2))
    Yx = Yx[:,0:8,:]

    Yy = tf.math.multiply(Yy, tf.expand_dims(tf.cast(tracks, dtype = 'float32'), axis = 2))
    Yy = Yx[:,0:8,:]
    
    Y = tf.concat([Yx, Yy], axis = 2)

    Y_final = tf.expand_dims(Y[:,0,:], axis=0)
    for i in range(7):
        Y_final = tf.concat([Y_final, tf.expand_dims(Y[:,i,:], axis=0)], axis = 0)

    return Y_final

## **Construct the LSTM model for the trajectory prediction part**

In [ ]:
def encoder_trajPred(num_states_steps, latent_dim):
    #The length of the large input vector. Depends on how many features we include.
    input_vec_length = 1280 
    num_static_features = 20000*4
    time_inputs = Input(shape=(input_vec_length, num_states_steps))
    final_points = Input(shape = (16,1))
    static_inputs = Input(shape=(num_static_features,1))
    a0 = Input(shape=(latent_dim,))
    c0 = Input(shape=(latent_dim,))
    a = a0
    c = c0

    # Create the encoder
    for t in range(num_states_steps):
        # Select the "t"th time step vector from enc_input. 
        x_time = Lambda(lambda z: z[:, :, t])(time_inputs)
        # Reshape x 
        x_time = Reshape((1, input_vec_length))(x_time)
        reshaped_fp = Reshape((1,16))(final_points)

        x_time = tf.concat([reshaped_fp, x_time], axis = 2)
        x_time = BatchNormalization()(x_time)                                 

        # LSTM_cell
        a, _, c = LSTM(latent_dim, kernel_initializer=GlorotUniform(), return_state = True)(x_time, initial_state=[a, c])

    #Convolve spatial features
    x_spatial = Conv1D(4, 5, strides = 2, padding = 'same', kernel_initializer=GlorotUniform())(static_inputs)
    x_spatial = MaxPool1D(pool_size=2, strides=2)(x_spatial)
    x_spatial = Conv1D(8, 5, strides = 2, padding = 'same', kernel_initializer=GlorotUniform())(x_spatial)
    x_spatial = MaxPool1D(pool_size=2, strides=2)(x_spatial)
    x_spatial = Conv1D(8, 5, strides = 2, padding = 'same', kernel_initializer=GlorotUniform())(x_spatial)
    x_spatial = MaxPool1D(pool_size=2, strides=2)(x_spatial)
    x_spatial = Conv1D(16, 5, strides = 2, padding = 'same', kernel_initializer=GlorotUniform())(x_spatial)
    x_spatial = MaxPool1D(pool_size=2, strides=2)(x_spatial)

    x_spatial = tf.keras.layers.Flatten()(x_spatial)
    x_spatial = Dropout(rate=DROPOUT_RATE_SPATIAL)(x_spatial)
    
    a = tf.keras.layers.Flatten()(a)
    x = tf.concat([a, x_spatial], axis = 1)
    output_encoder = Dense(LATENT_DIM, kernel_initializer=GlorotUniform())(x)

    return time_inputs, static_inputs, final_points, a0, c0, output_encoder

In [ ]:
def decoder_trajPred(output_encoder, outputs, num_future_steps, latent_dim):
    a0_dec = output_encoder
    c0_dec = Input(shape=(latent_dim,))
    a = a0_dec
    c = c0_dec
    zero_input = Input(shape=(128, 2, 2))

    for t in range(int(num_future_steps/10)):
        zi_t = Lambda(lambda z: z[:, t, :, :])(zero_input)
        zi_t = Reshape((1, 2*2))(zi_t)
        a, _, c = LSTM(latent_dim, kernel_initializer=GlorotUniform(), return_state = True)(zi_t, initial_state=[a, c])
        out_t = Dense(160, kernel_initializer=GlorotUniform())(a)
        outputs.append(out_t)
  
    return c0_dec, zero_input

In [ ]:
def TrajPredModel(num_states_steps, latent_dim, num_future_steps):
      
      time_inputs, static_inputs, final_points, a0, c0, output_encoder = encoder_trajPred(num_states_steps, latent_dim)

      outputs = []
      c0_dec, zero_input = decoder_trajPred(output_encoder, outputs, num_future_steps, latent_dim)

      model = Model(inputs=[time_inputs, static_inputs, final_points, a0, c0, c0_dec, zero_input], outputs=outputs)
      return model

## **Helper functions for training**

In [ ]:
def train_1_epoch_trajPred(epoch, train_dataset):
  logits_visualize, center_x_visualize, center_y_visualize, y_gt_visualize = 0, 0, 0, 0
  
  # Set the batch losses to be an empty list to only contain losses from this epochs batches.
  train_batch_losses = []

  # Iterate over the batches of the dataset.
  for step, batch in enumerate(train_dataset):

      # Send in 1 batch to the function train_step batch.
      # 'batch' is a DICTIONARY containing the past position, velocity, current position,
      # velocity etc. etc for all the objects in the scenario for all the time indices for
      # all batches. 
      train_batch_time, train_batch_static, y_batch_train, center_x, center_y = const_train_set(batch, get_trajectory_gt)
      bs = train_batch_time.shape[0]
      a0 = np.zeros((bs, LATENT_DIM))
      c0 = np.zeros((bs, LATENT_DIM))
      c0_dec = np.zeros((bs, LATENT_DIM))
      zero_input = np.zeros((bs, 128, 2, 2))

      with tf.GradientTape() as tape:
          # Run the forward pass of the layer.
          final_points = get_gt_final(batch, normalize(batch))
          logits = traj_model([train_batch_time, train_batch_static, final_points, a0, c0, c0_dec, zero_input], training=True)
          
          # Compute the loss value for this minibatch.
          loss_value = loss_fn(y_batch_train, logits)

      # Use the gradient tape to automatically retrieve
      # the gradients of the trainable variables with respect to the loss.
      grads = tape.gradient(loss_value, traj_model.trainable_weights)

      # Run one step of gradient descent by updating
      # the value of the variables to minimize the loss
      optimizer.apply_gradients(zip(grads, traj_model.trainable_weights))

      train_batch_losses.append(loss_value)

  # Prints the training loss after each epoch
  if len(train_batch_losses) != 0:
      print("The mean training loss after %d epochs: %.4f"% (epoch, float(sum(train_batch_losses)/len(train_batch_losses))))
      traj_train_epoch_losses.append(float(sum(train_batch_losses)/len(train_batch_losses)))

In [ ]:
def calculate_validation_loss(valid_dataset):
    logits_visualize_valid, center_x_valid_visualize, center_y_valid_visualize, y_gt_visualize_valid = 0,0,0,0
    valid_batch_losses = []

    # Calculates and prints the validation loss
    for step, batch in enumerate(valid_dataset):
      valid_time_batch, valid_static_batch, y_batch_valid, center_x_valid, center_y_valid = const_train_set(batch, get_trajectory_gt)
      bs = valid_time_batch.shape[0]
      a0 = np.zeros((bs, LATENT_DIM))
      c0 = np.zeros((bs, LATENT_DIM))
      c0_dec = np.zeros((bs, LATENT_DIM))
      zero_input = np.zeros((bs, 128, 2, 2))

      with tf.GradientTape() as tape:
        # Run the forward pass of the layer.
        final_points = get_gt_final(batch, normalize(batch))
        logits = traj_model([valid_time_batch, valid_static_batch, final_points, a0, c0, c0_dec, zero_input], training=False)
        
        # Compute the loss value for this minibatch.
        loss_value = loss_fn(y_batch_valid, logits)

      valid_batch_losses.append(loss_value)

    if len(valid_batch_losses) != 0:
        print("The mean validation loss after %d epochs: %.4f"% (epoch, float(sum(valid_batch_losses)/len(valid_batch_losses))))
        traj_valid_epoch_losses.append(float(sum(valid_batch_losses)/len(valid_batch_losses)))

## **Train the model on the Trajectory prediction task**

In [ ]:
epochs = EPOCHS

traj_train_epoch_losses = []
traj_valid_epoch_losses = []

traj_model = TrajPredModel(11, LATENT_DIM, 80)

In [ ]:
lr = START_LR_TRAJ
optimizer = Adam(learning_rate=lr, beta_1=0.9, beta_2=0.999)
loss_fn = tf.keras.losses.MeanSquaredError()

In [ ]:
for epoch in range(epochs):
    print('\nStart of epoch %d' % (epoch,))
    
    train_1_epoch_trajPred(epoch, train_dataset)
    calculate_validation_loss(valid_dataset)

In [ ]:
plt.plot(np.arange(0, len(traj_train_epoch_losses)), traj_train_epoch_losses)
plt.plot(np.arange(0, len(traj_valid_epoch_losses)), traj_valid_epoch_losses)
plt.show()

# **Test the system**

In [ ]:
# get the test dataset
test_dataset

In [ ]:
def testing(test_dataset): 
  test_batch_ade = [] 
  test_batch_fde = []
  for step, batch in enumerate(test_dataset):
    test_time_batch, test_static_batch, y_batch_test, center_x_test, center_y_test = const_train_set(batch, get_trajectory_gt)
    _, _, y_batch_test_end_point, _, _ = const_train_set(batch, get_gt_final)
    bs = test_time_batch.shape[0]
    a0 = np.zeros((bs, LATENT_DIM))
    c0 = np.zeros((bs, LATENT_DIM))
    c0_dec = np.zeros((bs, LATENT_DIM))
    zero_input = np.zeros((bs, 128, 2, 2))

    with tf.GradientTape() as tape:
      predicted_final_points = model([test_time_batch, test_static_batch, a0, c0], training=False)
      predicted_trajectories = traj_model([test_time_batch, test_static_batch, predicted_final_points, a0, c0, c0_dec, zero_input], training=False)
            
    # Compute the loss value for this minibatch.
    ade = np.sqrt(loss_fn(y_batch_test, predicted_trajectories))
    test_batch_ade.append(ade)

    fde = np.sqrt(loss_fn(y_batch_test_end_point, predicted_final_points))
    test_batch_fde.append(fde)
  return test_batch_ade, test_batch_fde

In [ ]:
test_batch_ade, test_batch_fde = testing(test_dataset)
print('Mean ade:', sum(test_batch_ade)/(len(test_batch_ade)))
print('Mean fde:',  sum(test_batch_fde)/(len(test_batch_fde)))